In [ ]:
"""
Homework 2 Part 1
Spam Filtering, Bayesian Net
CS 344 - Prof. Vander Linden
Taek Soo Nam
8th March 2019
"""



ang_corpus = [["Ang", "Gi", "Motti", "Motti", "Gi", "Ang"], ["Ang", "spam","kawaii", "spam", "very", "Gimotti"]]
ham_corpus = [["do", "i", "like", "green","eggs", "and", "ham"], ["i", "do"]]
corpus_combined = ["Ang", "Gi", "Motti", "Motti", "Gi", "Ang", "Ang", "spam","kawaii", "spam", "very", "Gimotti", "do", "i",
                    "like", "green", "eggs", "and", "ham", "i", "do"]
combined_corpus = list(dict.fromkeys(corpus_combined))

g = {}
for i in ham_corpus:
    for j in i:
        if j not in g:
            g[j] = 1
        else:
            g[j] += 1

b = {}
for i in ang_corpus:
    for j in i:
        if j not in b:
            b[j] = 1
        else:
            b[j] += 1

probability = {}
for word in corpus_combined:
    good = 0;
    bad = 0;
    if word in g:
        good = 2 * g[word]
    if word in b:
        bad = b[word]
    if good + bad >= 1:
        probability[word] = max(0.1, min(.99, min(1.0, bad / len(ang_corpus)) /(min(1.0, bad / len(ang_corpus)) + min(1.0, good / len(ham_corpus)))))

print("Combined corpus words and their probability:")
for word in probability:
    print(word, probability[word])

def filter(message):
    product = 1.0
    comp_product = 1.0

    for token in message:
        if token in probability:
            prob = probability[token]
        else:
            prob = 0.4
        product *= prob
        comp_product *= (1.0 - prob)

    return product / (product + comp_product)


spam1 = ["Ang", "Gi", "Motti", "Ang", "Gimotti"]
spam2 =  ["Kawaii", "spam", "Gimotti", "Ang", "I", "am"]
spam3 =  ["I", "Gimotti", "green", "eggs", "not", "spam"]
notspam = ["i", "like", "green", "eggs", "and", "ham"]

print("\nProbability of Ang Gimotti being spam is:", filter(spam1))

print("\nBy", filter(spam2), "'Kawaii spam Gimotti Ang I am' is spam.")

print("\nWhen word Gimotti goes in, the likelihood of an email being spam is", filter(spam3))

print("\nthis sentence is not spam", filter(notspam))





In [ ]:
"""
Homework 2 Part 2
Spam Filtering, Bayesian Net
CS 344 - Prof. Vander Linden
Taek Soo Nam
8th March 2019
"""

from probability import BayesNet, enumeration_ask, elimination_ask, gibbs_ask


T, F = True, False
grass = BayesNet([
    ('Cloudy', '', 0.5),
    ('Sprinkler', 'Cloudy', {T:0.1, F:0.5}),
    ('Rain', 'Cloudy', {T:0.8, F:0.2}),
    ('WetGrass', 'Sprinkler Rain', {(T, T):0.99, (T,F):0.9, (F,T):0.9, (F,F):0.0}),
    ])

print("\n\nProblem 2")
print("P(Cloudy)" + enumeration_ask('Cloudy', dict(), grass).show_approx())
print("P(Sprinkler|Cloudy)" + enumeration_ask('Sprinkler', dict(Cloudy=T), grass).show_approx())
print("Cloudy | the sprinkler is running and it's not raining)" + enumeration_ask('Cloudy', dict(Sprinkler=T, Rain=F), grass).show_approx())
print("WetGrass | it's cloudy, the sprinkler is running and it's raining)" + enumeration_ask('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), grass).show_approx())
print("Cloudy | the grass is not wet) " + enumeration_ask('Cloudy', dict(WetGrass=F), grass).show_approx())


Compute the number of independent values in the full joint probability distribution for this domain. Assume that no conditional independence relations are known to hold between these values.

Part c

Compute the number of independent values in the Bayesian network for this domain. Assume the conditional independence relations implied by the Bayes network.

i. P(Cloudy) = <0.5, 0.5>
ii. P(Sprinkler|cloudy) = <0.1, 0.9>
iii. P(Cloudy| the sprinkler is running and it’s not raining) = alpha * <0.1*0.2*0.5, 0.5*0.8*0.5>
                               			       = alpha * <0.01, 0.2>
                              			        = <0.0476, 0.9524>
iv. P(WetGrass|it’s cloudy, the sprinkler is running and it’s raining) = alpha * <0.99*0.1*0.8*0.5, 0.01*0.1*0.8*0.5>
                                    				   = <0.99, 0.01>
v. P(Cloudy|the grass is not wet) = alpha * sum( P(Cloudy^sprinkler^rain^-wetgrass) )
                     	     = alpha * sum( P(Cloudy) * P(sprinkler^rain^-wetgrass|Cloudy) )
                       	   = alpha * <0.5 * (0.1*0.8*0.01 + 0.1*0.2*0.1 + 0.9*0.8*0.1 + 0.9*0.2*1), 0.5 * (0.5*0.2*0.01 + 0.5*0.8*0.1 + 0.5*0.2*0.1 + 0.5*0.8*1)>
                       	   = alpha * <0.5*0.2548, 0.5*0.451>
                       	   = alpha * <0.1274, 0.2255>
                       	   = <0.361, 0.639>

i. P(Cloudy) = <0.5, 0.5>
ii. P(Sprinkler|Cloudy) = <0.1, 0.9>
iii. P(Cloudy|the sprinkler is running and it's not raining) = alpha * <0.1*0.2*0.5, 0.5*0.8*0.5>
                               	                      = alpha * <0.01, 0.2>
                              	                       = <0.0476, 0.9524>
iv. P(WetGrass|it's cloudy, the sprinkler is running and it's raining) = alpha * <0.99*0.1*0.8*0.5, 0.01*0.1*0.8*0.5>
                                      				 = <0.99, 0.01>
v. P(Cloudy|the grass is not wet) = alpha * sum( P(Cloudy^sprinkler^rain^-wetgrass) )
                      	    = alpha * sum( P(Cloudy) * P(sprinkler^rain^-wetgrass|Cloudy) )
                       	   = alpha * <0.5 * (0.1*0.8*0.01 + 0.1*0.2*0.1 + 0.9*0.8*0.1 + 0.9*0.2*1), 0.5 * (0.5*0.2*0.01 + 0.5*0.8*0.1 + 0.5*0.2*0.1 + 0.5*0.8*1)>
                       	   = alpha * <0.5*0.2548, 0.5*0.451>
                       	   = alpha * <0.1274, 0.2255>
                       	   = <0.361, 0.639>